<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
data = pd.read_csv('/datasets/insurance.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 5000 entries, 0 to 4999

Data columns (total 5 columns):

 #   Column             Non-Null Count  Dtype  

---  ------             --------------  -----  

 0   Пол                5000 non-null   int64  

 1   Возраст            5000 non-null   float64

 2   Зарплата           5000 non-null   float64

 3   Члены семьи        5000 non-null   int64  

 4   Страховые выплаты  5000 non-null   int64  

dtypes: float64(2), int64(3)

memory usage: 195.4 KB


In [3]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
#Разделим данные на признаки и цели
features = data.drop('Страховые выплаты', axis=1)
features.insert(0, 'Единицы', 1)
target = data['Страховые выплаты']

Данные содержат 5000 строк по 5 значений в каждой, пропуски отсутствуют. Столбец "Страховые выплаты" является целевым для обучения модели.

## Умножение матриц

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

**Обоснование:** 
Обозначения:
- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $Y$ — вектор целевого признака

- $P$ — обратимая матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания в исходной задаче:

$
a = Xw
$

Если умножить исходную матрицу признаков $X$ на обратимую матрицу $P$, то после повторного обучения модели будет получен новый вектор весов линейной регрессии $w'$. Таким образом после преобразования предсказания примут следующий вид:

$
a' = (XP)(w')
$


$
w' = ((XP)^T XP)^{-1} (XP)^T y = (P^T X^T X P)^{-1} (XP)^T y
$

$
w' = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$

$
(P^T)^{-1} P^T = E
$

$
w' = P^{-1} (X^T X)^{-1} X^T y
$

$
w = (X^T X)^{-1} X^T y
$

Таким образом:

$
w = P^{-1} \cdot w'
$

## Алгоритм преобразования

**Алгоритм**

- Получим случайную обратимую матрицу необходимого порядка.
- Умножим признаки на обратимую матрицу. 
- Повторно обучим алгоритм.

**Обоснование**

Значения признаков изменятся, а результату линейно регрессии останутся прежними.

## Проверка алгоритма

In [5]:
#Генерация обратимой матрицы
ncols = features.shape[1]
deter = 0
while deter == 0 :
    P = np.random.randint(100, size=(ncols,ncols))
    deter = np.linalg.det(P)

In [6]:
features_umn = features @ P

In [7]:
#Раздели данные на обучающую и валидационную выборки

features1_train, features1_valid, target1_train, target1_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)

features2_train, features2_valid, target2_train, target2_valid = train_test_split(
    features_umn, target, test_size=0.25, random_state=12345)

In [8]:
model1 = LinearRegression()
model1.fit(features1_train, target1_train)
score = r2_score(target1_valid, model1.predict(features1_valid))
print(score)

0.4352275712702657


In [9]:
model2 = LinearRegression()
model2.fit(features2_train, target2_train)
score = r2_score(target2_valid, model2.predict(features2_valid))
print(score)

0.4352275712702627


Качество модели по метрике r2 не изменилось.

**Вывод:**
Умножение матрицы признаков на обратимую матрицу не оказывает влияния на модель линейно регрессии. В ходе выполнения работы данный тезис был получен теоретически и проверен практически.

**Общий вывод:**
Данные содержат 5000 строк по 5 значений в каждой, пропуски отсутствуют. Столбец "Страховые выплаты" является целевым для обучения модели.

Теоретически доказано, что умножение матрицы признаков на обратимую матрицу не повлияет на модель линейной регрессии.

Проверено умножением случайной обратимой матрицы на матрицу признаков, значение матрики R2 не изменилось.